In [ ]:
#Creating Zeropi Hamiltonioan via Discretization 
import scipy.sparse as sps

EJ: torch.Tensor
EL: torch.Tensor
ECJ: torch.Tensor
EC: torch.Tensor
dEJ: torch.Tensor
dCJ: torch.Tensor
flux: torch.Tensor
ng: float
ncut: float
truncated_dim: float



In [14]:
import scipy.sparse as sps
import numpy as np
import torch

def scipy_sparse_to_torch_sparse(matrix):
    """Converts a scipy.sparse matrix to a PyTorch sparse tensor."""
    if not sps.isspmatrix_coo(matrix):
        matrix = matrix.tocoo()

    values = matrix.data
    indices = np.vstack((matrix.row, matrix.col))

    i = torch.LongTensor(indices)
    v = torch.FloatTensor(values)
    shape = matrix.shape

    return torch.sparse.FloatTensor(i, v, torch.Size(shape))

ECj = torch.tensor(5., requires_grad=True)
ECs = torch.tensor(5., requires_grad=True)
dCj = torch.tensor(5., requires_grad=True)
dEj = torch.tensor(5., requires_grad=True)
EJ = torch.tensor(5., requires_grad=True)
ng =1
phi_ext = 0.5
varphi_ext =0.5

Nphi = 100
Ntheta = 100

eye_Nphi = sps.eye(Nphi)
eye_Ntheta = sps.eye(Ntheta)

partial_phi_fd = scipy_sparse_to_torch_sparse(sps.kron(eye_Ntheta, sps.diags([-1, 1, 1], [0, 1, -Nphi+1], shape=(Nphi, Nphi))))

partial_phi_bk = scipy_sparse_to_torch_sparse(sps.kron(eye_Ntheta, sps.diags([1, -1, -1], [0, -1, Nphi-1], shape=(Nphi, Nphi))))

partial_theta_fd = scipy_sparse_to_torch_sparse(sps.kron(eye_Nphi, sps.diags([-1, 1, 1], [0, 1, -Ntheta+1], shape=(Ntheta, Ntheta))))

partial_theta_bk = scipy_sparse_to_torch_sparse(sps.kron(eye_Nphi, sps.diags([1, -1, -1], [0, -1, Ntheta-1], shape=(Ntheta, Ntheta))))


phi = np.linspace(0, 2 * np.pi, Nphi)
cos_phi = np.cos(phi)
cos_phi_m = np.diag(cos_phi)
sin_phi_adj = np.sin(phi-phi_ext/2)
sin_phi_adj_m = np.diag(sin_phi_adj)
phi_m = np.diag(phi)
_cos_phi = torch.tensor(cos_phi_m)
_phi = torch.tensor(phi_m)
_sin_phi_adj_m  = torch.tensor(sin_phi_adj_m)

theta = np.linspace(0, 2 * np.pi, Ntheta)
cos_theta_adj = np.cos(theta-varphi_ext/2)
sin_theta = np.sin(theta)
cos_theta_adj_m = np.diag(cos_theta_adj)
sin_theta_adj_m = np.diag(sin_theta)
_cos_theta_adj_m = torch.tensor(cos_theta_adj_m)
_sin_theta_m = torch.tensor(sin_theta)

#What do the krons do? The partial derivative operators are identical  - why would be chose different N to make them different? 

H = -2 * ECj * (partial_phi_fd * partial_phi_bk) \
    + 2 * ECs * ((1j * partial_theta_fd - ng) ** 2) \
    + 2 * ECs * dCj * partial_phi_fd * partial_theta_fd \
    - 2 * EJ * _cos_phi * _cos_theta_adj_m \
    + EL * _phi ** 2 \
    + 2 * EJ \
    + EJ * dEj * _sin_theta_m * _sin_phi_adj_m

RuntimeError: add(sparse, dense) is not supported. Use add(dense, sparse) instead.